# Selected Economic Characteristics: Commuting to Work from the American Community Survey

**[Work in progress]**

This notebook downloads [selected economic characteristics (DP03)](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03) from the American Community Survey 2018 5-Year Data.

Data source: [American Community Survey 5-Year Data 2018](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Authors: Peter Rose (pwrose@ucsd.edu), Ilya Zaslavsky (zaslavsk@sdsc.edu)

In [1]:
import os
import pandas as pd
from pathlib import Path
import time

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


## Download selected variables

* [Selected economic characteristics for US](https://data.census.gov/cedsci/table?tid=ACSDP5Y2018.DP03)

* [List of variables as HTML](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03.html) or [JSON](https://api.census.gov/data/2018/acs/acs5/profile/groups/DP03/)

* [Description of variables](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2018_ACSSubjectDefinitions.pdf)

* [Example URLs for API](https://api.census.gov/data/2018/acs/acs5/profile/examples.html)

### Specify variables from DP03 group and assign property names

Names must follow the [Neo4j property naming conventions](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-naming-rules-and-recommendations).

In [4]:
variables = {# COMMUTING TO WORK
             'DP03_0018E': 'workers16YearsAndOver',
             'DP03_0019E': 'droveAloneToWorkInCarTruckOrVan',
             'DP03_0019PE': 'droveAloneToWorkInCarTruckOrVanPct',
             'DP03_0020E': 'carpooledToWorkInCarTruckOrVan',
             'DP03_0020PE': 'carpooledToWorkInCarTruckOrVanPct',
             'DP03_0021E': 'publicTransportToWork',
             'DP03_0021PE': 'publicTransportToWorkPct',
             'DP03_0022E': 'walkedToWork',
             'DP03_0022PE': 'walkedToWorkPct',
             'DP03_0023E': 'otherMeansOfCommutingToWork',
             'DP03_0023PE': 'otherMeansOfCommutingToWorkPct',
             'DP03_0024E': 'workedAtHome',
             'DP03_0024PE': 'workedAtHomePct',
             'DP03_0025E': 'meanTravelTimeToWorkMinutes',
            }

In [5]:
fields = ",".join(variables.keys())

In [6]:
for v in variables.values():
    if 'Pct' in v:
        print('c.' + v + ' = toFloat(row.' + v + '),')
    else:
        print('c.' + v + ' = toInteger(row.' + v + '),')

c.workers16YearsAndOver = toInteger(row.workers16YearsAndOver),
c.droveAloneToWorkInCarTruckOrVan = toInteger(row.droveAloneToWorkInCarTruckOrVan),
c.droveAloneToWorkInCarTruckOrVanPct = toFloat(row.droveAloneToWorkInCarTruckOrVanPct),
c.carpooledToWorkInCarTruckOrVan = toInteger(row.carpooledToWorkInCarTruckOrVan),
c.carpooledToWorkInCarTruckOrVanPct = toFloat(row.carpooledToWorkInCarTruckOrVanPct),
c.publicTransportToWork = toInteger(row.publicTransportToWork),
c.publicTransportToWorkPct = toFloat(row.publicTransportToWorkPct),
c.walkedToWork = toInteger(row.walkedToWork),
c.walkedToWorkPct = toFloat(row.walkedToWorkPct),
c.otherMeansOfCommutingToWork = toInteger(row.otherMeansOfCommutingToWork),
c.otherMeansOfCommutingToWorkPct = toFloat(row.otherMeansOfCommutingToWorkPct),
c.workedAtHome = toInteger(row.workedAtHome),
c.workedAtHomePct = toFloat(row.workedAtHomePct),
c.meanTravelTimeToWorkMinutes = toInteger(row.meanTravelTimeToWorkMinutes),


In [7]:
print(len(variables.keys()))

14


## Download county-level data using US Census API

In [8]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=county:*'

In [9]:
df = pd.read_json(url_county, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,DP03_0018E,DP03_0019E,DP03_0019PE,DP03_0020E,DP03_0020PE,DP03_0021E,DP03_0021PE,DP03_0022E,DP03_0022PE,DP03_0023E,DP03_0023PE,DP03_0024E,DP03_0024PE,DP03_0025E,state,county
1,1520,741,48.8,313,20.6,14,0.9,311,20.5,74,4.9,67,4.4,8.2,02,195
2,2038,100,4.9,89,4.4,2,0.1,840,41.2,950,46.6,57,2.8,6.7,02,158
3,1976339,1507253,76.3,217539,11.0,41442,2.1,29957,1.5,49571,2.5,130577,6.6,26.0,04,013
4,12365,9848,79.6,1681,13.6,41,0.3,257,2.1,133,1.1,405,3.3,20.5,04,009


##### Add column names

In [10]:
df = df[1:].copy() # skip first row of labels
columns = list(variables.values())
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

Remove Puerto Rico (stateFips = 72) to limit data to US States

TODO handle data for Puerto Rico (GeoNames represents Puerto Rico as a country)

In [11]:
df.query("stateFips != '72'", inplace=True)

Save list of state fips (required later to get tract data by state)

In [12]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [13]:
df.head()

,workers16YearsAndOver,droveAloneToWorkInCarTruckOrVan,droveAloneToWorkInCarTruckOrVanPct,carpooledToWorkInCarTruckOrVan,carpooledToWorkInCarTruckOrVanPct,publicTransportToWork,publicTransportToWorkPct,walkedToWork,walkedToWorkPct,otherMeansOfCommutingToWork,otherMeansOfCommutingToWorkPct,workedAtHome,workedAtHomePct,meanTravelTimeToWorkMinutes,stateFips,countyFips
1,1520,741,48.8,313,20.6,14,0.9,311,20.5,74,4.9,67,4.4,8.2,02,195
2,2038,100,4.9,89,4.4,2,0.1,840,41.2,950,46.6,57,2.8,6.7,02,158
3,1976339,1507253,76.3,217539,11.0,41442,2.1,29957,1.5,49571,2.5,130577,6.6,26.0,04,013
4,12365,9848,79.6,1681,13.6,41,0.3,257,2.1,133,1.1,405,3.3,20.5,04,009
5,17036,14404,84.6,1324,7.8,54,0.3,237,1.4,357,2.1,660,3.9,22.8,04,023


In [14]:
# Example data
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')]

,workers16YearsAndOver,droveAloneToWorkInCarTruckOrVan,droveAloneToWorkInCarTruckOrVanPct,carpooledToWorkInCarTruckOrVan,carpooledToWorkInCarTruckOrVanPct,publicTransportToWork,publicTransportToWorkPct,walkedToWork,walkedToWorkPct,otherMeansOfCommutingToWork,otherMeansOfCommutingToWorkPct,workedAtHome,workedAtHomePct,meanTravelTimeToWorkMinutes,stateFips,countyFips
897,1603486,1223159,76.3,138748,8.7,46506,2.9,46313,2.9,36799,2.3,111961,7.0,26.0,06,073


In [15]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Admin2'

### Save data

In [16]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03CommutingAdmin2.csv", index=False)

## Download zip-level data using US Census API

In [17]:
url_zip = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=zip%20code%20tabulation%20area:*'

In [18]:
df = pd.read_json(url_zip, dtype='str')
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,DP03_0018E,DP03_0019E,DP03_0019PE,DP03_0020E,DP03_0020PE,DP03_0021E,DP03_0021PE,DP03_0022E,DP03_0022PE,DP03_0023E,DP03_0023PE,DP03_0024E,DP03_0024PE,DP03_0025E,state,zip code tabulation area
1,947,906,95.7,11,1.2,17,1.8,0,0.0,6,0.6,7,0.7,41.3,51,23833
2,563,563,100.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,38.1,51,23850
3,5645,5018,88.9,287,5.1,42,0.7,35,0.6,73,1.3,190,3.4,25.2,51,23851
4,1484,1282,86.4,58,3.9,11,0.7,32,2.2,54,3.6,47,3.2,36.1,51,23890


##### Add column names

In [19]:
df = df[1:].copy() # skip first row
columns = list(variables.values())
columns.append('stateFips')
columns.append('postalCode')
df.columns = columns

In [20]:
df.head()

,workers16YearsAndOver,droveAloneToWorkInCarTruckOrVan,droveAloneToWorkInCarTruckOrVanPct,carpooledToWorkInCarTruckOrVan,carpooledToWorkInCarTruckOrVanPct,publicTransportToWork,publicTransportToWorkPct,walkedToWork,walkedToWorkPct,otherMeansOfCommutingToWork,otherMeansOfCommutingToWorkPct,workedAtHome,workedAtHomePct,meanTravelTimeToWorkMinutes,stateFips,postalCode
1,947,906,95.7,11,1.2,17,1.8,0,0.0,6,0.6,7,0.7,41.3,51,23833
2,563,563,100.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,38.1,51,23850
3,5645,5018,88.9,287,5.1,42,0.7,35,0.6,73,1.3,190,3.4,25.2,51,23851
4,1484,1282,86.4,58,3.9,11,0.7,32,2.2,54,3.6,47,3.2,36.1,51,23890
5,120,120,100.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,16.8,51,23302


In [21]:
# Example data
df.query("postalCode == '90210'")

,workers16YearsAndOver,droveAloneToWorkInCarTruckOrVan,droveAloneToWorkInCarTruckOrVanPct,carpooledToWorkInCarTruckOrVan,carpooledToWorkInCarTruckOrVanPct,publicTransportToWork,publicTransportToWorkPct,walkedToWork,walkedToWorkPct,otherMeansOfCommutingToWork,otherMeansOfCommutingToWorkPct,workedAtHome,workedAtHomePct,meanTravelTimeToWorkMinutes,stateFips,postalCode
24526,8489,6386,75.2,491,5.8,91,1.1,123,1.4,139,1.6,1259,14.8,25.9,06,90210


In [22]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'PostalCode'

### Save data

In [23]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03CommutingZip.csv", index=False)

## Download tract-level data using US Census API
Tract-level data are only available by state, so we need to loop over all states.

In [24]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get={fields}&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    time.sleep(1)
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = list(variables.values())
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [25]:
df = pd.concat((get_tract_data(state) for state in stateFips))
df.fillna('', inplace=True)

In [26]:
df['tract'] = df['stateFips'] + df['countyFips'] + df['tract']

In [27]:
df['source'] = 'American Community Survey 5 year'
df['aggregationLevel'] = 'Tract'

In [28]:
# Example data for San Diego County
df[(df['stateFips'] == '06') & (df['countyFips'] == '073')].head()

,workers16YearsAndOver,droveAloneToWorkInCarTruckOrVan,droveAloneToWorkInCarTruckOrVanPct,carpooledToWorkInCarTruckOrVan,carpooledToWorkInCarTruckOrVanPct,publicTransportToWork,publicTransportToWorkPct,walkedToWork,walkedToWorkPct,otherMeansOfCommutingToWork,otherMeansOfCommutingToWorkPct,workedAtHome,workedAtHomePct,meanTravelTimeToWorkMinutes,stateFips,countyFips,tract,source,aggregationLevel
6,1252,876,70.0,110,8.8,98,7.8,56,4.5,72,5.8,40,3.2,18.6,06,073,06073008339,American Community Survey 5 year,Tract
7,3644,2971,81.5,363,10.0,42,1.2,9,0.2,141,3.9,118,3.2,26.5,06,073,06073008347,American Community Survey 5 year,Tract
8,5529,4445,80.4,650,11.8,47,0.9,43,0.8,135,2.4,209,3.8,23.6,06,073,06073008354,American Community Survey 5 year,Tract
9,3234,2596,80.3,163,5.0,121,3.7,52,1.6,26,0.8,276,8.5,20.7,06,073,06073008505,American Community Survey 5 year,Tract
10,3462,2417,69.8,229,6.6,17,0.5,133,3.8,214,6.2,452,13.1,24.8,06,073,06073017604,American Community Survey 5 year,Tract


### Save data

In [29]:
df.to_csv(NEO4J_IMPORT / "03a-USCensusDP03CommutingTract.csv", index=False)

In [30]:
df.shape

(73056, 19)